# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

## 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
# pipでオリジナルの解答に必要なライブラリーをインストール
#!pip install --upgrade pip
#!pip install -U pandas numpy scikit-learn imbalanced-learn

# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
# ↓こちらのライブラリは非推奨
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_category = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')
df_store = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv')
df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')

<ipython-input-2-d53d1135c37c>:24: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')


#■My memo  
★スタンス
未経験からデータサイエンティストになるために、  
「まずは構造化データの加工・集計」が基本  
分析可能な状態でデータを用意できなければ、分析の土俵に立つことすらできない。  
良い分析のためには、データの下ごしらえが必要。  

起業では様々な業務がシステム化されており、  
各システムの中では構造化データが生成・蓄積されており、  
本来構造化データはコンピュータで扱いやすく、  
分析しやすいデータとなりますが、  
それでも自由にデータを加工・集計できる人材の不足などにより、  
十分に活用されていないのが実態。

データ加工・集計の実践スキルを鍛える環境については、  
・取り組むべきお題があること  
・お題に対して対応方針を自身で考える機会があること  
・その対応方針に基づいて自身でプログラミングを実践できる環境があること  

★演習問題の進め方  
応用力・一般化力をつけるために  
①他人に解説できるレベルに理解する  
②問題を解いている最中の自分の思考回路をメモする

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

#先頭：head()
#末尾：tail()

#大量にデータがある場合、いきない全件取得すると負荷がかかるので、データの確認などは、件数を絞って中身を確認するのがよい。

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

#取得する項目が1つの場合は文字列で、複数の場合はリストで指定する
#必要な項目だけを選択することで、集計時間の短縮やコスト削減の観点で重要

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
#df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001"')
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']][df_receipt['customer_id'] == 'CS018205000001']

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id == "CS018205000001" & amount >= 1000')

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']].query('customer_id=="CS018205000001" & (amount>=1000 | quantity>=5)')

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" & (amount>=1000 & amount<=2000)')

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
#df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" & product_cd!="P071401019"')

df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].query('customer_id=="CS018205000001" & product_cd!="P071401012"')
#顧客ID（customer_id）が"CS018205000001を指定した場合、商品コード（product_cd）がP071401019が存在していなかったため、練習のためにP071401012に自分で変更

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')
#確認用に出力

In [ ]:
df_store.query('prefecture_cd != "13" & floor_area < 900')
df_store[(df_store['prefecture_cd'] != '13') & (df_store['floor_area'] < 900)]

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
# 最初に考えたコード
# df_store[df_store['store_cd'] == 'S14???'].sample(10)

# 次に考えたコード
# df_store[
#     df_store['store_cd'].str.contains('S14')
# ].sample(10)

df_store[df_store['store_cd'].str.startswith('S14')].head(10)

#df_store['store_cd'].str　store_cdを文字列としてするためにstrとしている

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer['customer_id'].str.endswith('1')].head(10)

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
# 最初に考えたコード
# df_store.query('address=="*横浜*"')

# 次に考えたコード
# df_store[
#     df_store['address'].str.startswith('神奈川県横浜市')
#     ]

#↑神奈川県が欠損していたらヒットしないのであまりよくない

df_store[
    df_store['address'].str.contains('横浜')
    ]

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
# df_customer[df_customer["status_cd"].str.startswith('A, B, C, D, E, F')].head(10)
# ↑なぜ違う？　→startswithは正規表現を読み取ることができない

df_customer[df_customer["status_cd"].str.contains('^[A-F]')].head(10)

#ポイント→^(先頭から)、[A-F](A-Fまで)
#^ハットは先頭文字であることを表す正規表現
#正規表現とは、複数の文字列を1つの記号で表す方法のこと

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer["status_cd"].str.contains('[1-9]$')].head(10)

#ポイント→[1-9](1-9まで)$(末尾)

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer[df_customer["status_cd"].str.contains('^[A-F]') & df_customer["status_cd"].str.contains('[1-9]$')].head(10)

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
# \d: 任意の数字（digit）に一致。
# \w: 任意の単語文字（word character）に一致。文字、数字、アンダースコアを含む。
# \s: 任意の空白文字（whitespace character）に一致。スペース、タブ、改行文字などを含む。
# .: 任意の文字に一致（改行文字を除く）。

df_store[df_store["tel_no"].str.contains(r'\d{3}-\d{3}-\d{4}')]

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
#df_customer.sort_values(by='birth_day', ascending=True).head(10)
df_customer.sort_values('birth_day', ascending=True).head(10)

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
#df_customer.sort_values(by='birth_day', ascending=False).head(10)
df_customer.sort_values('birth_day', ascending=False).head(10)

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
# rank()メソッドでランクをつける。
#"なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。"⇒method='min'
df_receipt["amount_rank"] = df_receipt["amount"].rank(ascending=False, method="min")
df_receipt.sort_values('amount_rank', ascending=True)[["customer_id", "amount", "amount_rank"]].head(10)

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
# method='first'で同一値（重複値）は登場順に順位付けされる。
df_receipt["amount_rank"] = df_receipt["amount"].rank(ascending=False, method="first")
df_receipt.sort_values('amount_rank', ascending=True)[["customer_id", "amount", "amount_rank"]].head(10)

#佐野先生解答↓amountをquantityで割って%を出して、それに対してランク付け
# df_receipt["amount_per_case"] = df_receipt["amount"] / df_receipt["quantity"]
# df_receipt["amount_per_case_rank"] = df_receipt["amount_per_case"].rank(ascending=False, method="first")
# df_receipt[["customer_id", "amount", "amount_per_case_rank"]].head()

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

#df_receipt.shape[0]→行数
#df_receipt.shape[1]→列数

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt['customer_id'].unique())

#df_receipt["customer_id"].nunique()

#.unique() メソッドは、その列の中で一意な値のリストを取得します。つまり、重複した値を取り除いたユニークな値のリストが得られます。
#.nunique() メソッドは、その列の中で一意な値の数を返します。つまり、重複した値を取り除いたユニークな値の数が得られます。

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).rename(columns={'amount': 'amount_by_storecd' , 'quantity': 'quantity_by_storecd'})

#df_receipt.groupby("store_cd").agg({"amount": "sum", "quantity": "sum"}).reset_index(drop=False)

#グループごとに値を求めることをAggregation→agg

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
#確認用に出力
#df_receipt[['customer_id', 'sales_ymd']].query('customer_id=="CS001114000005"')
df_receipt[['customer_id', 'sales_ymd']][df_receipt['customer_id'] =="CS001114000005"]
#確認結果、P-024の問題は、CS001114000005のsales_ymdが20190731ならOK

In [ ]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)
#reset_index()がないと、形が変わる。

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd': min}).head(10)


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
#問題意訳
#来店の最新日と最古日が違うということは、2日以上店舗を利用している顧客。
#つまりリピーターなので、今後も継続的に店舗を利用してくれる可能性のある顧客たち

df_new = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()

# df_new[df_new[('sales_ymd', 'max')] != df_new[('sales_ymd', 'min')]].head(10)

df_new.columns = ['customer_id', 'sales_ymd_max', 'sales_ymd_min']
df_new.query('sales_ymd_max != sales_ymd_min').head(10)

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index().sort_values('amount', ascending=False).head(5)

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
#中央値=データを順番に並べた時に中央に位置するデータのこと
#平均値は外れ値に引っ張られる性質があるので、平均値だけでなく中央値も見る必要がある。

df_receipt.groupby('store_cd').agg({'amount':'median'}).sort_values('amount', ascending=False).head(5)

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
#確認用に出力
df_receipt[['store_cd', 'product_cd']].query('store_cd=="S14006"')

In [ ]:
#感覚的にこうしたいけど、これは誤り↓
#df_receipt.groupby('store_cd').agg({'product_cd':'mode'}).sort_values('amount', ascending=False).head(5)
#'SeriesGroupBy' object has no attribute 'mode'
#"'SeriesGroupBy'オブジェクトに'mode'属性はありません"

In [ ]:
#問題意訳→店舗ごとで一番売れている商品を算出する
#最頻値を使うことで、例えば、どの商品が人気かといった傾向を掴むことができる

#appply
#applyに関数を渡すと、その処理が各グループに適用される

#lambda
#lambda式は、無名関数を定義する
#通常の関数内にがっつり処理を書くほどではないけど、
#処理を簡単にまとめておきたい場合に使っている
#lambda式を使うと、簡単な処理を一行でまとめて書く事ができる
#for文を回してリストに要素を追加したい場合に用いる内包表記に似ている

# df.groupby('grouping_content').最頻を求めたい対象.apply(lambda x: x.mode())

# ↓解1
# df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index()

# ↓解2
# f = lambda x: x.mode()
# df_receipt.groupby('store_cd').product_cd.apply(f).reset_index()

# ↓解3
df_receipt.groupby('store_cd').apply(lambda x: x['product_cd'].mode()).reset_index()

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
# 標本→母集団からいくつかのサンプルを抽出したもの
# 標本分散とは、「標本から計算した分散」を示しています。つまり対象の全体から一部を抽出したデータの分散を意味している
# 分散とは、「データが平均からどれだけ散らばっているか」示しており、データのばらつきが大きいと分散が大きくなり、ばらつきが小さくなると分散は0に近づく特徴がある

#問題意訳
#店舗ごとの売上金額のうち、平均を基準にばらつきの大きい5つを表示せよ

# 引数ddofはdata degree of freedom(デルタ自由度)の略
# var(ddof=自由度)

# 店舗コード（store_cd）ごとの売上金額（amount）の標本分散(var(ddof=0))
# ddof=0であれば、与えられたデータを集合全体と見てその分散を求める標本分散を求められる
# ddof=1であれば、一部のデータから全体のデータの分散を計算する不偏分散を求められる
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount', ascending=False).head(5)

In [ ]:
#P-30：確認用
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).rename(columns={'amount': 'amount_by_storecd' , 'quantity': 'quantity_by_storecd'}).reset_index().sort_values('amount_by_storecd', ascending=True).head(5)
df_receipt.groupby('store_cd').agg({'amount': 'sum', 'quantity': 'sum'}).rename(columns={'amount': 'amount_by_storecd' , 'quantity': 'quantity_by_storecd'}).reset_index().sort_values('amount_by_storecd', ascending=False).head(5)

In [ ]:
#P-30：標本分散ではなく不偏分散
df_receipt.groupby('store_cd').amount.var(ddof=1).reset_index().sort_values('amount', ascending=False).head(5)

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
# 店舗コード（store_cd）の売上金額（amount）の標本標準偏差(std(ddof=0))
# reset_indexでインデックスを振り直し、ソートする(sort_values)、降順(ascending=False)

#P-30で扱った平方根が標準偏差
#分んは計算過程で値を二乗しているのに対し、その平方根である標準偏差を求めることでデータの単位を実際のデータに揃えられる

#PandasとNumpyでddofのデフォルト値が異なることに注意
#std-standard deviation

df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index().sort_values('amount', ascending=False).head(5)

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
#P-032:確認用
df_receipt.amount.min()

In [ ]:
#P-032:確認用
df_receipt.amount.median()

In [ ]:
# 売上金額（amount）をパーセンタイル値(quantile)
# np.arange(5)/4 >>> [0, 0.25, 0.5, 0.75, 1]
# 「パーセンタイル値」とは、全体を100として小さい方から数えて何番目になるのかを示す数値で、50パーセンタイルが中央値
# 「30パーセンタイルの人」は「100人のうち小さいほうから数えて30番目」ということ

#arrange(start, stop)
#arrangeを使うとstart～end-1の範囲で連番のリストを取得でき、リストに対して演算を行うと各要素に演算結果が反映される


df_receipt.amount.quantile(q=np.arange(5)/4).reset_index().rename(columns={'index': 'percentile value'})

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [22]:
#1件あたりの平均売上金額が一定以上の店舗に限定して分析を行える
# 店舗コード（store_cd）ごとにグループ分け。
# 売上金額（amount）の平均(mean)を算出し、reset_index()でインデックスを振り直す
# query()メソッドで売上金額（amount）の300以上
df_receipt.groupby('store_cd').amount.mean()[df_receipt.groupby('store_cd').amount.mean() >= 330].reset_index()

,store_cd,amount
0,S12013,330.194130
1,S13001,348.470386
2,S13003,350.915519
3,S13004,330.943949
4,S13015,351.111960
5,S13019,330.208616
6,S13020,337.879932
7,S13052,402.867470
8,S14010,348.791262
9,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
# ~はNot。、顧客ID（customer_id）がZから始まる(str.startswith("Z"))
# 顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均
df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
# 顧客IDが"Z"を除外。'customer_id'ごとにグループを分ける。（2547.742234529256）全顧客の平均
amount_mean = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()

# 顧客ID（customer_id）ごとに売上金額（amount）を合計する
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()

# amount_mean(全顧客の平均)以上の１０件を表示
df_amount_sum[df_amount_sum['amount'] >= amount_mean].head(10)


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
#顧客情報を持つマスタデータ(店舗データフレーム)と購買履歴を持つデータ(レシート明細データフレーム)を
#結合することで、「どのような属性の人に、どのような購買履歴があるのか」を見ることが出来る

#結合の種類
#pd.merge(df1, df2, how = inner/left/right/outer, on = ”結合キー”)
#内部結合	inner
#左外部結合	left
#右外部結合	right
#完全外部結合	outer

# merge(A(df), B(df), how='inner'(内部結合), on='共通する列')
pd.merge(df_receipt, df_store[['store_cd','store_name']], how='inner', on='store_cd').head(10)


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
# 商品データフレームの方に、カテゴリーIDがあるけど、カテゴリーIDだけ見れても、
# 何の商品か分からず扱いづらいので、
# カテゴリーデータフレームにあるカテゴリー名を商品データフレームにくっつけたい

# merge(A(df), B(df), how='inner'(内部結合), on='共通する列')
pd.merge(df_product
         , df_category[['category_small_cd','category_small_name']]
         , how='inner', on='category_small_cd').head(10)

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
#vlookupみたいな感じで、AにあるキーがBにもあれば、B'を持ってくる。なければ0を表示みたいな

# まず、df_customerにおいて、性別コード1であり、顧客IDがZではないものを抽出する(df_customer_senzaMen)
df_customer_senzaMen = df_customer.query('gender_cd == 1 and not customer_id.str.startswith("Z")')
# 次に、df_receiptにおいて、各顧客ごとの売上金額を合計する(df_amount_sum)
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
# ↑2つをマージ。fillna(0)で買い物の実績がない顧客については売上金額を0を表示
pd.merge(df_customer_senzaMen['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).head(10)


---★あとで復習
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
df_data = df_receipt[(~df_receipt['customer_id'].str.startswith("Z"))]

df_cnt = df_data[~df_data.duplicated(subset=['customer_id', 'sales_ymd'])] \
         .groupby('customer_id').sales_ymd.count().reset_index() \
         .sort_values('sales_ymd', ascending=False).head(20)

#duplicated(subset=重複を排除する項目)
#↑売上日数の多い顧客の上位20件を求める際、購入日をそのままカウントすると購入回数になってしまうので、
#購入日のユニークな件数をカウントする必要がある
#groupbyをする前にduplicatedを使って重複の削除を行う

df_sum = df_data.groupby('customer_id').amount.sum().reset_index() \
         .sort_values('amount', ascending=False).head(20)

pd.merge(df_cnt, df_sum, how='outer', on='customer_id')

In [ ]:
#別解
df_data = df_receipt[~df_receipt['customer_id'].str.startswith("Z")]

df_cnt = df_data.groupby('customer_id')['sales_ymd'].nunique().reset_index() \
         .sort_values('sales_ymd', ascending=False).head(20)

df_sum = df_data.groupby('customer_id')['amount'].sum().reset_index() \
         .sort_values('amount', ascending=False).head(20)

pd.merge(df_cnt, df_sum, how='outer', on='customer_id')

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

直積とは、2つのデータにおけるレコードの組み合わせを全通り列挙したデータのこと  
これを取得するのがクロス結合  
時系列のデータの欠損を補う際などに使える

In [ ]:
df_store_count = len(df_store)
df_product_count = len(df_product)

df_store_count * df_product_count

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
df_recipt2 = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_recipt2['difference'] = df_recipt2['amount'] - df_recipt2['amount'].shift(1)
df_recipt2.head(10)

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
df_recipt2 = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_recipt2['1日前の売上年月日'] = df_recipt2['sales_ymd'].shift(1)
df_recipt2['1日前の売上合計'] = df_recipt2['amount'].shift(1)
df_recipt2['2日前の売上年月日'] = df_recipt2['sales_ymd'].shift(2)
df_recipt2['2日前の売上合計'] = df_recipt2['amount'].shift(2)
df_recipt2['3日前の売上年月日'] = df_recipt2['sales_ymd'].shift(3)
df_recipt2['3日前の売上合計'] = df_recipt2['amount'].shift(3)
df_recipt2.head(10)

In [ ]:
#練習
#df_recipt2 = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
#df_recipt2['+3daysago'] = df_recipt2['amount'] + df_recipt2['amount'].shift(1)+ df_recipt2['amount'].shift(2)+ df_recipt2['amount'].shift(3)
#df_recipt2.head(10)

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

横持ちのでーたには結果をそのまま理解したりグラフ化したりといったことがしやすいという利点があるため、  
縦持ちのデータを横持ちに変換したいことがある。

In [ ]:
#10歳ごとの階級の算出の仕方

#パターン1
#if文を使う
#if 10～19なら"10代"みたいな
df_customer['age_range'] = df_customer['age'].apply(lambda x:
    '10代' if 10 <= x <= 19 else
    '20代' if 20 <= x <= 29 else
    '30代' if 30 <= x <= 39 else
    '40代' if 40 <= x <= 49 else
    '50代' if 50 <= x <= 59 else
    '60代' if 60 <= x <= 69 else
    '70代' if 70 <= x <= 79 else
    '80代' if 80 <= x <= 89 else
    '90代' if 90 <= x <= 99 else 'その他'
)

df_customer.head(10)

#↑だと100歳以上に対して不適切

#パターン2
#年齢を10で割り、小数点以下を切り捨てた上で10をかけるという流れで算出する
#ex. 19 / 10 =1.9 →小数点以下を切り捨てると1 →1に10を掛けると10 →19歳は10代



In [ ]:
# pd.merge(df_receipt, df_customer, how='outer').groupby(['gender' , 'age']).amount.sum().reset_index()

# 顧客データフレームとレシート明細データフレームをcutomer_idをキーとして内部結合
df_sales = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_sales['age_group'] = (df_sales['age'] // 10) * 10
df_sales_summary = df_sales.groupby(['age_group', 'gender'])['amount'].sum().reset_index()
df_sales_summary['gender'].replace({0: '男性', 1: '女性', 9: '不明'}, inplace=True)

#ピボットテーブルの活用
#pivot_table(データフレーム,index=行に割り当てる項目,columns=列に割り当てる項目,value=値として配置する項目,aggfunc=集計方法)
#indexに指定した項目の種類の数だけ行数ができて、columnsに指定した項目の種類の数だけ列数ができる
df_sales_summary_pivot = df_sales_summary.pivot_table(index='age_group', columns='gender', values='amount').reset_index()
df_sales_summary_pivot.columns = ['年代', '性別不明の売上金額', '女性の売上金額', '男性の売上金額']
df_sales_summary_pivot.fillna(0, inplace=True)
df_sales_summary_pivot

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
#最初に考えたコード
df_sales_summary_pivot = df_sales_summary.pivot_table(index='gender', columns='age_group', values='amount').reset_index()
df_sales_summary_pivot.fillna(0, inplace=True)
df_sales_summary_pivot

#↑性別を縦持ちし、3項目に変換というのは、列が「年代」「性別」「売上金額」になるように変換するということ
#→年代と性別を総当たりにするということ

In [ ]:
df_sales = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_sales['age_group'] = (df_sales['age'] // 10) * 10
df_sales_summary = df_sales.groupby(['age_group', 'gender'])['amount'].sum().reset_index()
df_sales_summary.replace({'女性':'01','男性':'00','不明':'99'})

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
df_customer['birth_day'] = df_customer['birth_day'].dt.strftime('%Y%m%d')
df_customer[['customer_id', 'birth_day']].head(10)

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'], format='%Y%m%d')
df_customer[['customer_id', 'application_date']].head(10)

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'], format='%Y%m%d')
df_receipt[['receipt_no', 'receipt_sub_no' , 'sales_ymd']].head(10)

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
#unix秒を変換する場合は引数としてunit=sを付ける

check = df_receipt.copy()
check["sales_epoch"] = pd.to_datetime(check["sales_epoch"], unit="s")
check[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
check = df_receipt.copy()
check["sales_epoch"] = pd.to_datetime(check["sales_epoch"], unit="s").dt.strftime('%Y')
check[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

In [ ]:
#↓確認用
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'],
                          unit='s').dt.year.rename('sales_year')],
          axis=1).head(10)

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
check = df_receipt.copy()
check["sales_epoch"] = pd.to_datetime(check["sales_epoch"], unit="s").dt.strftime('%m')
check[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
check = df_receipt.copy()
check["sales_epoch"] = pd.to_datetime(check["sales_epoch"], unit="s").dt.strftime('%d')
check[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
# check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
# check["amount_cate"] = (check["amount"] > 2000) + 0
# check

#applyとlambda式を組み合わせて、条件式の真偽により値を変える
#lambda x: 真の場合の値 if xを含む条件式 else 偽の場合の値

df_receipt2 = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby('customer_id')['amount'].sum().reset_index()
df_receipt2['0or1'] = df_receipt2['amount'].apply(lambda x: 1 if x >= 2000 else 0)
df_receipt2.head(10)

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
#佐野先生の答え
check = df_customer.copy()
check["postal_cd_first"] = df_customer["postal_cd"].apply(lambda x: int(x.split("-")[0]))
check["tokyo_flg"] = ((check["postal_cd_first"] >= 100) & (check["postal_cd_first"] <= 209)) + 0
pd.merge(df_receipt, check, how="inner", on="customer_id").groupby("tokyo_flg").agg({"customer_id": "nunique"})

In [ ]:
#↓splitではなく、スライシングを使う
#文字列[開始位置：終了位置]とすることで、文字列に対し、開始位置～終了位置満の範囲で部分文字列を取り出せる
#取り出した部分文字列で範囲指定を行う為に、intで数値に変換する


df_tmp = df_customer[['customer_id', 'postal_cd']].copy()

df_tmp['postal_flg'] = df_tmp['postal_cd']. \
                        apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

pd.merge(df_tmp, df_receipt, how='inner', on='customer_id'). \
    groupby('postal_flg').agg({'customer_id':'nunique'})

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
df_customer_tmp = df_customer[['customer_id', 'address']].copy()

df_customer_tmp['prefecture_cd'] = \
            df_customer['address'].str[0:3].map({'埼玉県': '11',
                                                '千葉県':'12',
                                                '東京都':'13',
                                                '神奈川':'14'})

df_customer_tmp.head(10)

In [ ]:
#↓佐野先生の回答 check["address_code"] = 10: address_code 列に全ての行に対して値 10 を代入
#(check["address"].str.contains("埼玉"))がtrueであれば、1を返す

check = df_customer.copy()
check["address_code"] = 10
check["address_code"] = check["address_code"] + (check["address"].str.contains("埼玉"))
check["address_code"] = check["address_code"] + (check["address"].str.contains("千葉")) * 2
check["address_code"] = check["address_code"] + (check["address"].str.contains("東京")) * 3
check["address_code"] = check["address_code"] + (check["address"].str.contains("神奈川")) * 4
check[["customer_id", "address", "address_code"]].head(10)

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
#quantile(項目,パーセンタイル)

df_sales_amount = df_receipt[['customer_id', 'amount']]. \
                        groupby('customer_id').sum().reset_index()

pct25 = np.quantile(df_sales_amount['amount'], 0.25)
pct50 = np.quantile(df_sales_amount['amount'], 0.5)
pct75 = np.quantile(df_sales_amount['amount'], 0.75)

def pct_group(x):
    if x < pct25:
        return 1
    elif pct25 <= x < pct50:
        return 2
    elif pct50 <= x < pct75:
        return 3
    elif pct75 <= x:
        return 4

df_sales_amount['pct_group'] = df_sales_amount['amount'].apply(pct_group)

df_sales_amount.head(10)

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
check = df_customer.copy()
check["age_cate"] = check["age"] // 10 * 10
check["age_cate"] = check["age_cate"].apply(lambda x: 60 if x > 60 else x)
check[["customer_id", "birth_day", "age_cate"]].head(10)

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
#+演算子を使って文字列を同士を連結することができる
#.astypeで数値から文字列に変換する

check["age_gender"] = check["age_cate"].astype(str) + "_" + check["gender"]
check[["customer_id", "birth_day", "age_cate", "age_gender"]].head(10)

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

カテゴリの値ごとに項目を分けて保持し、データが該当する項目に1を、  
それ以外の項目に0を付与する。  
このようにして作成した変数をダミー変数と呼ぶ。  
カテゴリ変数から多変量解析の説明変数や機械学習の特徴量を作成するときに行われる処理  
値の種類(0:男性、1:女性、9:不明)に応じた項目列を設け、  
該当するものに1、それ以外に0を付与する

In [ ]:
df_customer['性別コード0:男性'] = df_customer['gender_cd'].apply(lambda x: 1 if x == 0 else 0)
df_customer['性別コード1:女性'] = df_customer['gender_cd'].apply(lambda x: 1 if x == 1 else 0)
df_customer['性別コード9:不明'] = df_customer['gender_cd'].apply(lambda x: 1 if x == 9 else 0)

df_customer[['customer_id','性別コード0:男性','性別コード1:女性','性別コード9:不明']].head(10)

In [ ]:
#get_dummies(データフレーム,ダミー変数化する項目)　を使うことで、ダミー変数を簡単に作成できる
#新しく作成される項目名は、　元の項目名_カテゴリ値　となる
#pd.get_dummies(df_customer[['customer_id', 'gender_cd']],culumns=['gender_cd'])
#↑gender_cdをダミー化する

check = df_customer.copy()
dummy_list = []
for i in check["gender_cd"].unique():
    dummy_list += ["gender_cd" + "_" + str(i)]
    check["gender_cd" + "_" + str(i)] = (check["gender_cd"] == i) + 0
check[["customer_id"] + dummy_list].head(10)

In [ ]:
#get_dummies(データフレーム,ダミー変数化する項目)　を使うことで、ダミー変数を簡単に作成できる
#新しく作成される項目名は、　元の項目名_カテゴリ値　となる
#pd.get_dummies(df_customer[['customer_id', 'gender_cd']],culumns=['gender_cd'])
#↑gender_cdをダミー化する
pd.get_dummies(df_customer[['customer_id', 'gender_cd']],
               columns=['gender_cd']).head(10)

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

顧客ごとの売上金額合計に対し、平均が0、分散・標準偏差が1になるように変換する  
この変換を標準化という  
各データから平均値を引き、標準偏差で割ることで、  
分布を平均0、分散1に近づけることができる

In [ ]:
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
amount_mean = check["amount"].mean()
amount_sd = check["amount"].std()
check["amount_norm"] = (check["amount"] - amount_mean) / amount_sd
check.head(10)

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

顧客ごとの売上金額合計に対し、最小値が0、最大値が1になるように変換する  
この変換を正規化という  
各データから最小値を引き、最大値-最小値で割ることで、データが0～1の範囲に収まるようにする

In [ ]:
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
amount_max = check["amount"].max()
amount_min = check["amount"].min()
check["amount_norm"] = (check["amount"] - amount_min) / (amount_max - amount_min)
check.head(10)

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

対数化はデータの値が小さい時はその範囲が拡大され、  
値が大きい時はその範囲が縮小される  


In [ ]:
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
check["amount_log"] = np.log10(check["amount"])
check.head(10)

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',
                                   engine='python'). \
                                   groupby('customer_id'). \
                                   agg({'amount':'sum'}).reset_index()
df_sales_amount['log_amount'] = np.log10(df_sales_amount['amount'] + 0.5)
df_sales_amount.head(10)

#売上金額合計が0となった場合への配慮として0.5を足している↑

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
#自然対数で対数化する場合、NumPyのlogを使用する
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
check["amount_log"] = np.log(check["amount"])
check.head(10)

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',
                                   engine='python'). \
                                   groupby('customer_id'). \
                                   agg({'amount':'sum'}).reset_index()

df_sales_amount['log_amount'] = np.log(df_sales_amount['amount'] + 0.5)

df_sales_amount.head(10)

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
check = df_product.copy()
check["profit"] = check["unit_price"] - check["unit_cost"]
check.head(10)

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
#(単価-原価) / 単価　=利益率

#欠損に対する四則演算の結果は欠損になる
check = df_product.copy()
check["profit_rate"] = (check["unit_price"] - check["unit_cost"]) / check["unit_price"]
check["profit_rate"].dropna().mean()

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
#(単価 - 原価) / 単価 =0.3 となるような単価を求める
#単価 = 原価 / 0.7
#

check = df_product.copy()
check["unit_price_new"] = (check["unit_cost"] / 0.7) // 1
check["profit_rate"] = (check["unit_price_new"] - check["unit_cost"]) / check["unit_price_new"]
check.head()

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
check = df_product.copy()
check["unit_price_new"] = round(check["unit_cost"] / 0.7)
check["profit_rate"] = (check["unit_price_new"] - check["unit_cost"]) / check["unit_price_new"]
check.head()

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
#numpyのceilで切り上げを行うことができる
#引数が少数の場合はその値より大きくかつ最小の整数を返す、引数が整数の場合はその値を返す

check = df_product.copy()
check = check[["product_cd", "unit_price", "unit_cost"]].dropna(how="any", axis=0)
check["unit_price_new"] = (check["unit_cost"] / 0.7).apply(math.ceil)
check["profit_rate"] = (check["unit_price_new"] - check["unit_cost"]) / check["unit_price_new"]
check.head()

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
#消費税込み金額を求めるので、掛ける値は1.1となる

check = df_product.copy()
check["unit_price_tax"] = (check["unit_price"] * 1.1) // 1
check.head(10)

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
check = pd.merge(df_receipt, df_product[["product_cd", "category_major_cd"]], how="inner", on="product_cd")
check.groupby("customer_id").agg({"amount": "sum"})
check

In [ ]:
check = pd.merge(df_receipt, df_product[["product_cd", "category_major_cd"]], how="inner", on="product_cd")
#レシート明細データフレームと、商品データフレームのプロダクトcdとカテゴリ大区分の列を、プロダクトキcdをキーに内部結合したものcheckというdfとする

check = pd.merge(
    check.groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False),
    check[check["category_major_cd"]==7].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False),
    how="inner", on="customer_id")
#↑で作成したcheckというデータフレームをカスタマーidごとにamountの合計を出す→amount_x
#カテゴリ大区分7で絞り込んだ状態で、amountの合計を出す→amount_y
#カスタマーidをキーとして上記を内部結合し、それをcheckというデータフレームとする

check["rate"] = check["amount_y"] / check["amount_x"]
check.head(10)

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
check = pd.merge(df_receipt, df_customer[["customer_id", "application_date"]], how="inner", on="customer_id")
#レシート明細データフレームと、顧客データフレームのカスタマーidと会員申込日を、カスタマーidをキーに内部結合したものcheckというdfとする

check = check.drop_duplicates(subset=["customer_id", "sales_ymd"])
#カスタマーidとsales_ymdの重複を取り除く

check["application_date"] = pd.to_datetime(check["application_date"], format="%Y%m%d")
check["sales_ymd"] = pd.to_datetime(check["sales_ymd"], format="%Y%m%d")
#日付型にデータ変換する→計算するために

check["day_diff"] = (check["sales_ymd"] - check["application_date"]).dt.days
#day_diffという列を追加する(売上日-申し込み日)

check[["customer_id", "sales_ymd", "application_date", "day_diff"]].head(10)

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
check["month_diff"] = (check["sales_ymd"].dt.year - check["application_date"].dt.year) * 12 + check["sales_ymd"].dt.month - check["application_date"].dt.month
#年数*12で月数を算出し、それに月数を足す
check[check["month_diff"] >= 1][["customer_id", "sales_ymd", "application_date", "month_diff"]].head(10)
#1か月未満は切り捨てで絞り込み

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
check["year_diff"] = check["month_diff"] // 12
check[check["year_diff"] >= 1][["customer_id", "sales_ymd", "application_date", "year_diff"]].head(10)

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
check["unixtime_diff"] = check["day_diff"] * 24 * 60 * 60
check[["customer_id", "sales_ymd", "application_date", "unixtime_diff"]].head(10)



---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
#日付型のデータxに対し、x.weekday()とすることで曜日を数値で取得することができる
#月曜日が0となる
#直前月曜日の日数は、日付型に変換した売上日からこの経過日数を引くことで求められる
#経過日数をrelativedeltaで日付に足し引きできる形に変換する

df_tmp =df_receipt[['sales_ymd']].copy()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['直近月曜日からの日数'] = df_tmp['sales_ymd'].apply(lambda x:x.weekday())
df_tmp['直近月曜日の日付'] = df_tmp['sales_ymd'].apply(lambda x: x-relativedelta(days=x.weekday()))
df_tmp.head(10)

In [ ]:
#SA
check = df_receipt.copy()
check["sales_ymd"] = pd.to_datetime(check["sales_ymd"], format="%Y%m%d")
check["sales_ymd_diff"] = check["sales_ymd"].dt.dayofweek
check["sales_ymd_cate"] = check["sales_ymd"].apply(lambda x: x - relativedelta(days=x.dayofweek))
check[["sales_ymd", "sales_ymd_cate", "sales_ymd_diff"]].head(10)

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(axis=0, frac=0.1, random_state=17).head(10)
#ランダムにサンプリングする都合上、
#処理後のイメージに記載されている結果と実際の結果は一致しないことがある

#sample(frac=サンプリング率)

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
check = df_customer.groupby("gender_cd").sample(frac=0.1)
check.groupby('gender_cd').agg({'customer_id' : 'count'})

#サンプリング前後で各カテゴリの割合は維持されることになる
#このようなサンプリングを層化抽出といい、
#各カテゴリの分布が大きくなる場合などでよく利用される方法
#層化抽出ではカテゴリのことを層と呼ぶ
#ここでは、性別コードの値が層にあたる

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
check[abs(check["amount"] - check["amount"].mean()) > check["amount"].std() * 3].head(10)
#absは絶対値を求める関数。絶対値は数値の符号を無視して、その値の大きさだけを取り出す。
#合計 - 平均値を引いて、3となるもの
#ここでは外れ値を平均から3σ以上離れたもの

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
check = df_receipt[df_receipt["customer_id"].str.contains("^[^Z]")].groupby("customer_id").agg({"amount": "sum"}).reset_index(drop=False)
iqr = check["amount"].quantile(0.75) - check["amount"].quantile(0.25)
check[
    (check["amount"] < check["amount"].quantile(0.25) - 1.5 * iqr) |
    (check["amount"] > check["amount"].quantile(0.75) + 1.5 * iqr)].head(10)

#NumPyのPercentileで第一四分位と第三四分位を計算し、この2つを引くことでIQR計算する
#第一四分位の値からIQRの1.5倍を引くことで下限値を、第三四分位の値にIQRの1.5倍を引くことで下限値を、
#第3四分位の値にIQRの1.5倍を足すことで上限値をそれぞれ求めている
#最後にqueryの中でこの上限値および下限値を売上金額合計と比較し、外れ値を抽出

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum(axis=0)

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product_1 = df_product.dropna(how="any", axis=0)
print(df_product.shape)
print(df_product_1.shape)

(10030, 6)
(10023, 6)


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

欠損値への対応には、  
・行を削除して対応⇒リストワイズ除去→欠損項目以外の項目が保有する情報もまとめて削除される  
・欠損値を予測値で補完して対応  
平均値や中央値や最頻値で補完

fillna(項目1： 欠損時の値1, 項目2： 欠損時の値2)  


In [3]:
df_product_2 = df_product.copy()

# 単価（unit_price）の欠損値を平均値で補完
mean_unit_price = round(df_product_2['unit_price'].mean())
df_product_2['unit_price'].fillna(mean_unit_price, inplace=True)

# 原価（unit_cost）の欠損値を平均値で補完
mean_unit_cost = round(df_product_2['unit_cost'].mean())
df_product_2['unit_cost'].fillna(mean_unit_cost, inplace=True)

# 結果を表示
df_product_2.head()
missing_values = df_product_2.isnull().sum()
#欠損がないかを確認するための↑
missing_values

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# 欠損値はfillnaを用いる
# np.roundで偶数よせの四捨五入。中央値は、np.nanmedianで補完する
#nanmedianは欠損(nan)のデータを除外した上で平均を計算する
#中央値は平均値よりも外れ値の影響を受けにくいので、外れ値のデータが多い場合には、中央値で補完するのがよい

df_product_3 = df_product.fillna({'unit_price':np.round(np.nanmedian(df_product['unit_price'])),
                                  'unit_cost':np.round(np.nanmedian(df_product['unit_cost']))})

# isnull().sum()で欠損を確認する
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [18]:
#df_product.groupby('category_small_cd').agg('median')

df_tmp = df_product.groupby('category_small_cd').agg(median_price=('unit_price', 'median'),median_cost=('unit_cost', 'median')).reset_index()
#各商品の小区分ごとに、単価と原価の中央値を算出し、df_tmpをデータフレームとする

df_product_4 = pd.merge(df_product, df_tmp, how='inner', on='category_small_cd')
#df_tmpとdf_product_4を'category_small_cd'をキーに内部結合する

df_product_4['unit_price'] = df_product_4[['unit_price', 'median_price']].apply(lambda x: np.round(x[1])if np.isnan(x[0]) else x[0], axis=1)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'median_cost']].apply(lambda x: np.round(x[1])if np.isnan(x[0]) else x[0], axis=1)
#もし x[0]（つまり unit_price 列の値）が NaN（欠損値）である場合、x[1]（つまり median_price 列の値）を四捨五入して返す
#それ以外の場合は、x[0]（つまり unit_price 列の値）をそのまま返す
#axis=1: apply() メソッドを行方向（列ごと）に適用することを指定する

df_product_4.isnull().sum()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,median_price,median_cost
0,P040101001,4,401,40101,198.0,149.0,283.0,212.5
1,P040101002,4,401,40101,218.0,164.0,283.0,212.5
2,P040101003,4,401,40101,230.0,173.0,283.0,212.5
3,P040101004,4,401,40101,248.0,186.0,283.0,212.5
4,P040101005,4,401,40101,268.0,201.0,283.0,212.5
...,...,...,...,...,...,...,...,...
10025,P091503001,9,915,91503,280.0,210.0,1080.0,810.0
10026,P091503002,9,915,91503,680.0,510.0,1080.0,810.0
10027,P091503003,9,915,91503,1080.0,810.0,1080.0,810.0
10028,P091503004,9,915,91503,1130.0,848.0,1080.0,810.0


In [ ]:
df_product_4 = df_product.copy()
# 単価（unit_price）と原価（unit_cost）それぞれの処理を行う
# fillnaを用いて欠損値を補完する。
# 各商品の小区分（category_small_cd）ごとに中央値を出す。transformを用いる。
for x in ['unit_price', 'unit_cost']:
    df_product_4[x] = df_product_4[x].fillna(df_product_4.groupby('category_small_cd')[x]
                                            .transform('median').round())
# isnull().sum()で欠損を確認する
df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

必要なデータは、  
①全顧客の全期間の売上合計  
②全顧客の2019年の売上合計  

②を①で割る  
＊販売実績のない場合は0として計算する  
＊割合が0超のものを抽出  

In [ ]:
df_receipt_all = df_receipt.groupby('customer_id').agg(amount_all=('amount', 'sum')).reset_index()
df_receipt_2019 = df_receipt.query('20190101 <= sales_ymd <= 20191231').groupby('customer_id').agg(amount_2019=('amount', 'sum')).reset_index()
df_sales_rate = df_customer[['customer_id']].merge(df_receipt_2019, how='left', on='customer_id').merge(df_receipt_all, how='left', on='customer_id')
df_sales_rate['amount_2019'] = df_sales_rate['amount_2019'].fillna(0)
df_sales_rate['amount_all'] = df_sales_rate['amount_all'].fillna(0)
df_sales_rate['amount_rate'] = df_sales_rate[['amount_2019','amount_all']].apply(lambda x: 0 if x[0] == 0 else x[0] / x[1], axis=1)
#0除算が発生する場合は割合を0とする↑
df_sales_rate['amount_rate'] = df_sales_rate['amount_rate'].fillna(0)

#売上割合が0を超えるものだけを抽出
df_sales_rate.query('amount_rate > 0').head(10)

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
df_geocode
#確認結果→longitude と latitudeは郵便番号ごとではなく、住所コードに保持している
#ので、郵便番号が同じでも、longitude と latitudeが異なる

#郵便番号をキーにするためには、郵便番号ごとにlongitude と latitudeの平均を用いる

In [99]:
df_geocode_1 = df_geocode.groupby('postal_cd').agg(m_longitude=('longitude', 'mean'),m_latitude=('latitude', 'mean')).reset_index()
df_customer_1 = pd.merge(df_customer, df_geocode_1,how='inner', on='postal_cd')
df_customer_1.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longitude,m_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,0-00000000-0,139.31779,35.41358
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,0-00000000-0,139.31779,35.41358
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,0-00000000-0,139.31779,35.41358
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,0-00000000-0,139.31779,35.41358
5,CS021314000093,江藤 美佐,1,女性,1986-06-03,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20151015,0-00000000-0,139.31779,35.41358
6,CS021413000049,大野 幸子,1,女性,1973-04-17,45,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150703,0-00000000-0,139.31779,35.41358
7,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
8,CS015415000209,大谷 倫子,1,女性,1970-11-25,48,136-0076,東京都江東区南砂**********,S13015,20150909,B-20090610-C,139.83502,35.67193
9,CS037614000045,長沢 麻緒,1,女性,1952-06-19,66,136-0076,東京都江東区南砂**********,S13037,20150209,5-20091207-6,139.83502,35.67193


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

佐野先生からのレクチャーメモ  

わざわざsin cosを使って地球の丸みを加味しようとしているが、これは意味ない  
4kmとか3kmとかの距離の計算に地球の丸みは加味する必要はない  
普通に三平方の定理で解いて、単位をkmに直す110.9を掛けるところは必要だが、  

言われた通りただ実装して無駄なパフォーマンスを発揮させるより、  
その差がどれくらいの影響があるのかを考えてプログラムを作成する

折角モデルの中身が分かるのに、たかだか0.1%の正しさを求めるために  
ブラックボックス化することが良いのか　考える  
モデル作成の際にも大事な考え方になる

In [102]:
check = pd.merge(df_customer_1,df_store[["store_cd","longitude","latitude","address"]],how="left",left_on="application_store_cd",right_on="store_cd")
check["distance_km"] = 110.9 * ((check["m_longitude"] - check["longitude"])**2 + (check["m_latitude"] - check["latitude"])**2)**0.5
check[["customer_id","address_x","address_y","distance_km"]].head(10)

,customer_id,address_x,address_y,distance_km
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
1,CS021303000023,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
2,CS021303000007,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
3,CS021313000183,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
4,CS021314000098,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
5,CS021314000093,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
6,CS021413000049,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.425763
7,CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.690972
8,CS015415000209,東京都江東区南砂**********,東京都江東区南砂二丁目,1.351467
9,CS037614000045,東京都江東区南砂**********,東京都江東区南砂一丁目,1.690972


申込店舗と顧客の住所の距離を求める  
平面距離は三平方の定理で求められる  
斜辺cの2乗は，他の辺a，bをそれぞれ2乗した数の和に等しい  
三平方の定理で2点間の緯度経度の差を求めて、111.1を掛けるとおおよその距離になる

In [ ]:
# 計算式の関数を作成
def calc_distance(x1, y1, x2, y2):
    x1_r = np.radians(x1)
    x2_r = np.radians(x2)
    y1_r = np.radians(y1)
    y2_r = np.radians(y2)
    return 6371 * np.arccos(np.sin(y1_r) * np.sin(y2_r)
                            + np.cos(y1_r) * np.cos(y2_r)
                            * np.cos(x1_r - x2_r))

# 緯度経度つき顧客データフレーム（df_customer_1）と店舗データフレーム（df_store）と結合
df_tmp = pd.merge(df_customer_1, df_store, how='inner',
                  left_on=['application_store_cd'], right_on=['store_cd'])

# 距離を求める
df_tmp['distance'] = calc_distance(df_tmp['m_longitude'], df_tmp['m_latitude'],
                                          df_tmp['longitude'], df_tmp['latitude'])
df_tmp[['customer_id', 'address_x', 'address_y', 'distance']].head(10)


# (別解) mathを使ったパターン
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2))
                       + math.cos(math.radians(y1)) * math.cos(math.radians(y2))
                            * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

df_tmp = pd.merge(df_customer_1, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')

df_tmp['distance'] =   df_tmp[['m_longitude', 'm_latitude','longitude', 'latitude']]. \
                                apply(lambda x: calc_distance(x[0], x[1], x[2], x[3]), axis=1)

df_tmp[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

必要なデータ  
①店舗ごとの購買履歴にある名前と住所→df_receiptから  
＊同一顧客データの中で売上金額が最も高いものを残す

In [ ]:
df_receipt_tmp = df_receipt.groupby('customer_id').agg(sum_amount=('amount','sum')).reset_index()
#カスタマーidごとのamountの合計を求める
df_customer_u = pd.merge(df_customer, df_receipt_tmp,how='left',on='customer_id')
#顧客データフレームとカスタマーidで内部結合
df_customer_u['sum_amount'] = df_customer_u['sum_amount'].fillna(0)
#sum_amountがnaのものは0とする
df_customer_u = df_customer_u.sort_values(['sum_amount', 'customer_id'],ascending=[False, True])
#ascendingをfalse,trueとしているのは、売上金額合計が降順、顧客IDが昇順となる

df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'],keep='first', inplace=True)
#重複判定に使われる項目を氏名と郵便番号にする。firstにすることで、最初に登場するデータが残る。
#inplaceをtrueにすることで、データフレームそのものを書き換える

df_customer_u

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

★統合名寄IDでグループ化することで、"実際の顧客単位"での売り上げ分析をすることができる

In [ ]:
# 顧客データフレーム（df_customer）と名寄せした名寄顧客データフレーム（df_customer_u）を内部結合する
# ※名前（customer_name）と郵便番号（postal_cd）が一致するものを結合する
df_customer_n = pd.merge(df_customer, df_customer_u[['customer_id', 'customer_name', 'postal_cd']],how='inner', on=['customer_name', 'postal_cd'])
# カラム名を変更
df_customer_n.rename(columns={'customer_id_x': 'customer_id', 'customer_id_y': 'integration_id'}, inplace=True)

df_customer_n

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [86]:
unique_customer_count = df_receipt['customer_id'].nunique()
unique_customer_count
#↑総数　これを8:2に分けるということは
#8307 * 0.8 = 6646と
#8307 * 0.2 = 1661に分けてみる
df_receipt.sample(6646)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
43253,20170224,1487894400,S13001,1152,1,ZZ000000000000,P070505040,1,268
76727,20170409,1491696000,S13039,1142,1,CS039604000004,P080804001,1,100
90300,20190713,1562976000,S12029,1142,2,ZZ000000000000,P071401001,1,1100
33800,20190314,1552521600,S13003,112,2,CS003212000041,P060503003,1,78
48707,20190625,1561420800,S13041,1122,1,CS041414000005,P090401004,1,88
...,...,...,...,...,...,...,...,...,...
39789,20170809,1502236800,S13020,112,2,CS020215000038,P070402002,1,175
99583,20180906,1536192000,S13044,1152,1,ZZ000000000000,P071401003,1,1100
16951,20190618,1560816000,S14027,1192,2,ZZ000000000000,P080402006,1,135
96740,20180925,1537833600,S13016,112,2,CS016215000007,P080202001,1,30


In [ ]:
# 売り上げ実績のある顧客データを作成(df_customerとdf_receiptを結合)
df_tmp = pd.merge(df_customer, df_receipt['customer_id'], how='inner', on='customer_id')

# df_customerを学習用データとテスト用データに分割　(train_test_split)を用いる
df_train, df_test = train_test_split(df_tmp, test_size=0.2, random_state=71)

# 学習用データとテスト用データそれぞれの割合を出す
print('学習データ割合: ', len(df_train) / len(df_tmp))
print('テストデータ割合: ', len(df_test) / len(df_tmp))


学習データ割合:  0.7999908650771901
テストデータ割合:  0.2000091349228099


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [23]:
# レシート明細データフレーム（df_receipt）をコピーする
df_tmp = df_receipt.copy()
# sales_ymdをint64からstr型へ変更し、スライスを用いて年月を取り出す
df_tmp['sales_ymd'] = df_tmp['sales_ymd'].astype('str').str[:6]

# sales_ymd(年月)でグループ分けして、売上金額（amount）を合計する
df_tmp = df_tmp.groupby('sales_ymd').agg({'amount': sum}).reset_index()

# 学習データとテストデータに分ける関数
def split_date(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start:test_start], df[test_start:test_start+test_size]

# モデル構築用データを3セット作成
df_train_1, df_test_1 = split_date(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
df_train_2, df_test_2 = split_date(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=1)
df_train_3, df_test_3 = split_date(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=2)
# df_train_2とdf_train_3の表示は割愛
df_train_1

,sales_ymd,amount
0,201701,902056
1,201702,764413
2,201703,962945
3,201704,847566
4,201705,884010
5,201706,894242
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739


---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

売上実績ありと売上実績なしを目的変数として学習させる場合、  
どちらかの値に偏りがある場合、学習にも偏りが出てしまうので、  
それぞれが同じ数のデータ数になるように減らしたり、増やしたりする。  
特に多数派のデータを減らして不均等を解消することを、アンダーサンプリングという

In [71]:
# レシート明細データフレーム（df_receipt）を顧客ごとにグループ分けし、売り上げを合計する
df_tmp = df_receipt.groupby('customer_id').agg({'amount': sum}).reset_index()
# 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）を結合(左結合)
df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')

# 売り上げ実績(amount)がある場合は１、ない場合は０を入れる
df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

# 売上実績のある顧客数と売上実績のない顧客数の件数を出力する
print('0の件数', len(df_tmp.query('buy_flg==0')))
print('1の件数', len(df_tmp.query('buy_flg==1')))

# RandomUnderSampler でアンダーサンプリングする
rs = RandomUnderSampler(random_state=71)
df_sample, _ = rs.fit_resample(df_tmp, df_tmp.buy_flg)

# 売上実績のある顧客数と売上実績のない顧客数の件数を出力する
print('0の件数', len(df_sample.query('buy_flg==0')))
print('1の件数', len(df_sample.query('buy_flg==1')))


0の件数 13665
1の件数 8306
0の件数 8306
1の件数 8306


In [25]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')

df_tmp['is_buy_flag'] = np.where(df_tmp['amount'].isnull(), 0, 1)

rs = RandomUnderSampler(random_state=71)

df_down_sampling, _ = rs.fit_resample(df_tmp, df_tmp.is_buy_flag)

print('0の件数', len(df_down_sampling.query('is_buy_flag == 0')))
print('1の件数', len(df_down_sampling.query('is_buy_flag == 1')))

0の件数 8306
1の件数 8306


---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [26]:
# 顧客データフレーム（df_customer）のgender_cdの重複を取り除く
df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()

# 性別のデータフレーム(df_gender)を作成したので顧客データフレームからgenderカラムを削除する
df_customer_s = df_customer.drop(columns='gender')


---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [27]:
# 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を結合する
df_product_full = pd.merge(df_product,df_category[['category_small_cd', 'category_major_name',
                                                   'category_medium_name', 'category_small_name']],
                                                   how='inner', on='category_small_cd')
df_product_full.head()


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

Googleドライブ、このファイルがあるフォルダに保存したい。  
マウントを行う↓

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [62]:
# to_csv(ファイル形式はCSV)を用いて出力する
df_product_full.to_csv('./Colab Notebooks/P-94test.csv', encoding='UTF-8', index=False)

# (別解)
# コード例2（BOM付きでExcelの文字化けを防ぐ）
# df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.csv',encoding='utf_8_sig',index=False)


---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [63]:
# to_csv(ファイル形式はCSV)を用いて出力する
df_product_full.to_csv('./Colab Notebooks/P-95test.csv', encoding='CP932', index=False)


---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [64]:
# to_csv(ファイル形式はCSV)を用いて出力する
df_product_full.to_csv('./Colab Notebooks/P-96test.csv', header=False ,encoding='UTF-8', index=False)


---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [68]:
# read_csvを用いて読み込む
df_tmp = pd.read_csv('./Colab Notebooks/P-94test.csv')
df_tmp.head(10)


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [67]:
# read_csvを用いて読み込む
df_tmp = pd.read_csv('./Colab Notebooks/P-96test.csv', header=None)
df_tmp.head(10)


,0,1,2,3,4,5,6,7,8
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [69]:
# df_product_full→to_csv(ファイル形式はCSV)を用いて出力する
# タブ文字\tで区切ったtsvファイルとして保存
df_product_full.to_csv('./Colab Notebooks/P-99test.csv', sep='\t', encoding='UTF-8', index=False)


---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [70]:
# TSVの読み込みはread_tableを用いる
df_tmp = pd.read_table('./Colab Notebooks/P-99test.csv', encoding='utf-8')
df_tmp.head(10)


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！